# Test draft strategies
## seems like delta ADP is the best strategy so far
* Have "agent" strategies and strategies for everyone else
* Can choose which metric to maximize on
* Can choose whether to draft highest in that metric, or the largest positional delta in that metric

In [63]:
import pandas as pd
import numpy as np

## Define Functions
### Build structures to contain outputs
### define strategy to maximize on a metric
### choose to maximize on the delta in that metric or the absolute highest of that metric
### define function to run the simulation for all drafting positions

In [76]:
def build_pick_frame(teams, rounds, frame):
    
    # get vectors length of teams and rounds
    round_vec = [x for x in range(1, rounds + 1)]
    team_vec = [x for x in range(1, teams + 1)]
    picks = rounds * teams

    ## create output dataframe for the simulation
    pick_frame = pd.DataFrame()
    pick = 1
    
    # get picking order, snake draft
    for rou in round_vec:
        if np.mod(rou, 2) == 1:
            round_order = team_vec
        else:
            round_order = list(reversed(team_vec))
        for tm in round_order:
            pick_frame.loc[len(pick_frame), 'Pick'] = pick
            pick_frame.loc[len(pick_frame) - 1, 'Round'] = rou
            pick_frame.loc[len(pick_frame) - 1, 'Team'] = tm
            pick = pick + 1
    
    # assign remaining dataframe, pick_frame
    remaining = frame
    remaining['Pick'] = ''
    remaining = remaining.sort_values('Overall').reset_index(drop = True)
    
    return pick_frame, remaining


def absolute_strategy(remaining, metric, metric_sort, pick_frame, pick, team):
    for i in range(len(remaining)):

        ###########################################
        ### this is where we select sorting strategy
        ###########################################


        ## select highest player in metric in position of need

        remaining_sorted = (remaining
                            .sort_values(metric
                                         , ascending = metric_sort)
                            .iloc[i])


        ########################################
        ## figure out position limits
        ########################################

        pos = remaining_sorted.pos

        if pos == 'QB':
            max_len = 1
            max_flex = 0
        elif pos == 'TE':
            max_len = 1
            max_flex = 6
        else:
            max_len = 2
            max_flex = 5


        num_of_pos = len(pick_frame
                         .loc[(pick_frame.Team == team) 
                              & (pick_frame.pos == pos)])
        num_of_flex = len(pick_frame
                         .loc[(pick_frame.Team == team) 
                              & (pick_frame.pos.isin(['RB', 'WR', 'TE']))])

        #######################################
        # conditions to draft player
        #######################################

        if (num_of_pos < max_len) | ((num_of_pos == max_len) 
           & (num_of_flex < max_flex) 
           & (pos in ['RB', 'WR', 'TE'])):
            pick_frame.loc[pick_frame.Pick == pick, 'Names'] = remaining_sorted.Names
            pick_frame.loc[pick_frame.Pick == pick, 'prediction_median'] = remaining_sorted.prediction_median
            pick_frame.loc[pick_frame.Pick == pick, 'Overall'] = remaining_sorted.Overall
            pick_frame.loc[pick_frame.Pick == pick, 'pts_next_year'] = remaining_sorted.pts_next_year
            pick_frame.loc[pick_frame.Pick == pick, 'pos'] = remaining_sorted.pos
            break

    # update players remaining
    remaining = (remaining
                 .loc[remaining.Names != remaining_sorted
                                          .Names])
    return remaining, pick_frame


def delta_strategy(remaining, metric, metric_sort, pick_frame, pick, team):
    # rank partition by position, select top 2 in each in metric
    remaining_temp = remaining
    remaining_temp['pos_rank_temp'] = remaining_temp.groupby('pos')[metric].rank(ascending = metric_sort)
    remaining_temp = remaining_temp.loc[remaining_temp.pos_rank_temp <= 2]
    group_temp = remaining_temp.groupby('pos')[metric].agg(np.ptp).reset_index()
    group_temp = group_temp.sort_values(metric, ascending = False)
    for i in range(len(remaining_temp)):
        try:
            pos_temp = group_temp.pos.iloc[i]
        except:
            print(group_temp)
        #print(pos_temp)

        if pos_temp == 'QB':
            max_len = 1
            max_flex = 0
        elif pos_temp == 'TE':
            max_len = 1
            max_flex = 6
        else:
            max_len = 2
            max_flex = 5


        num_of_pos = len(pick_frame
                         .loc[(pick_frame.Team == team) 
                              & (pick_frame.pos == pos_temp)])
        num_of_flex = len(pick_frame
                         .loc[(pick_frame.Team == team) 
                              & (pick_frame.pos.isin(['RB', 'WR', 'TE']))])

        #######################################
        # conditions to draft player
        #######################################

        if (num_of_pos < max_len) | ((num_of_pos == max_len) 
           & (num_of_flex < max_flex) 
           & (pos in ['RB', 'WR', 'TE'])):
            #print(remaining_temp.loc[(remaining_temp.pos == pos_temp)])
            try:
                row_of_interest = (remaining_temp
                                   .loc[(remaining_temp.pos == pos_temp)]
                                  .iloc[0])
            except IndexError:
                print('error')
                # in a tie breaker, pick random
                row_of_interest = (remaining_temp
                                   .loc[(remaining_temp.pos == pos_temp), 0]
                                  .iloc[0])
            #print(row_of_interest)
            pick_frame.loc[pick_frame.Pick == pick, 'Names'] = row_of_interest.Names
            pick_frame.loc[pick_frame.Pick == pick, 'prediction_median'] = row_of_interest.prediction_median
            pick_frame.loc[pick_frame.Pick == pick, 'Overall'] = row_of_interest.Overall
            pick_frame.loc[pick_frame.Pick == pick, 'pts_next_year'] = row_of_interest.pts_next_year
            pick_frame.loc[pick_frame.Pick == pick, 'pos'] = row_of_interest.pos
            break
        #else:
         #   pass
    # update players remaining
    remaining = (remaining
                 .loc[remaining.Names != row_of_interest
                                          .Names])
    return remaining, pick_frame

def run_simulation_one_year(teams, rounds, frame, agent_strategy, main_strategy):
    agent_outcome = []

    for agent_team_number in range(1, teams + 1):

        # create dataframe of players, picks for output and sorting
        pick_frame, remaining = build_pick_frame(teams, rounds, frame)
        
        team = 1

        # cycle through all picks in the draft
        for pick in range(1, len(pick_frame) + 1):

            if pick == 1:
                pick_frame['pos'] = ''

            # select team to pick
            team = pick_frame.loc[pick_frame.Pick == pick].Team.iloc[0]
            #print(team)
            
            # select team strategy
            if team == agent_team_number:
                metric = agent_strategy['maximizing_metric']
                strategy = agent_strategy['sub_strategy']
            else:
                metric = main_strategy['maximizing_metric']
                strategy = main_strategy['sub_strategy']
            if metric == 'Overall':
                metric_sort = True
            if metric == 'prediction_median':
                metric_sort = False

            # cycle through picks in this way for absolute strategy
            if strategy == 'absolute':
                remaining, pick_frame = absolute_strategy(remaining, metric, metric_sort, pick_frame, pick, team)
                #print(pick_frame)
                
            ## select highest marginal score difference based on model in position of need
            if strategy == 'delta':
                remaining, pick_frame = delta_strategy(remaining, metric, metric_sort, pick_frame, pick, team)
                #print(pick_frame)


        # report outcomes
        simulation_outcome = (pick_frame
                              .groupby('Team')
                              .sum()
                              .reset_index())
        #print(simulation_outcome)

        simulation_outcome['finish'] = simulation_outcome['pts_next_year'].rank()

        (agent_outcome
         .append(simulation_outcome
                 .loc[simulation_outcome.Team == agent_team_number, 'finish'].iloc[0]))
    return agent_outcome, simulation_outcome, pick_frame
            

In [77]:
#######################################################
####################### set parameters ###########


## read in output from RF models

for year in [2014, 2015, 2016, 2017, 2018]:
    frame = pd.DataFrame()
    for pos in ['RB', 'WR', 'QB', 'TE']:
        temp = pd.read_csv('simulation_{}_{}.csv'.format(pos, year))
        temp['pos'] = pos
        frame = (frame
                 .append(temp)
                 .sort_values('Overall')
                 .reset_index(drop = True)
                )
        
    rounds = 7
    teams = 16

    main_strategy = {'maximizing_metric': 'Overall'
                    , 'sub_strategy': 'absolute'}

    agent_strategy = {'maximizing_metric': 'Overall'
                     , 'sub_strategy': 'delta'}

    agent_outcome, simulation_outcome, pick_frame = run_simulation_one_year(teams, rounds, frame, agent_strategy, main_strategy)
    print(year)
    print(np.average(agent_outcome))
    print('\n')

/Users/JoshCoven/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


2014
11.3125


2015
9.0


2016
10.375


2017
4.75


2018
6.375




In [66]:
#######################################################
####################### set parameters ###########


## read in output from RF models

for year in [2016, 2017, 2018]:
    frame = pd.DataFrame()
    for pos in ['RB', 'WR', 'QB', 'TE']:
        temp = pd.read_csv('simulation_{}_{}.csv'.format(pos, year))
        temp['pos'] = pos
        frame = (frame
                 .append(temp)
                 .sort_values('Overall')
                 .reset_index(drop = True)
                )
        
    rounds = 7
    teams = 10

    main_strategy = {'maximizing_metric': 'Overall'
                    , 'sub_strategy': 'absolute'}

    agent_strategy = {'maximizing_metric': 'Overall'
                     , 'sub_strategy': 'absolute'}

    agent_outcome, simulation_outcome, pick_frame = run_simulation_one_year(teams, rounds, frame, agent_strategy, main_strategy)
    print(year)
    print(np.average(agent_outcome))
    print('\n')

2016
5.5


2017
5.5


2018
5.5




In [67]:
#######################################################
####################### set parameters ###########


## read in output from RF models

for year in [2016, 2017, 2018]:
    frame = pd.DataFrame()
    for pos in ['RB', 'WR', 'QB', 'TE']:
        temp = pd.read_csv('simulation_{}_{}.csv'.format(pos, year))
        temp['pos'] = pos
        frame = (frame
                 .append(temp)
                 .sort_values('Overall')
                 .reset_index(drop = True)
                )
        
    rounds = 7
    teams = 10

    main_strategy = {'maximizing_metric': 'Overall'
                    , 'sub_strategy': 'absolute'}

    agent_strategy = {'maximizing_metric': 'prediction_median'
                     , 'sub_strategy': 'delta'}

    agent_outcome, simulation_outcome, pick_frame = run_simulation_one_year(teams, rounds, frame, agent_strategy, main_strategy)
    print(year)
    print(np.average(agent_outcome))
    print('\n')

/Users/JoshCoven/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


2016
4.7


2017
5.3


2018
4.8




In [68]:
#######################################################
####################### set parameters ###########


## read in output from RF models

for year in [2016, 2017, 2018]:
    frame = pd.DataFrame()
    for pos in ['RB', 'WR', 'QB', 'TE']:
        temp = pd.read_csv('simulation_{}_{}.csv'.format(pos, year))
        temp['pos'] = pos
        frame = (frame
                 .append(temp)
                 .sort_values('Overall')
                 .reset_index(drop = True)
                )
        
    rounds = 7
    teams = 10

    main_strategy = {'maximizing_metric': 'Overall'
                    , 'sub_strategy': 'delta'}

    agent_strategy = {'maximizing_metric': 'prediction_median'
                     , 'sub_strategy': 'delta'}

    agent_outcome, simulation_outcome, pick_frame = run_simulation_one_year(teams, rounds, frame, agent_strategy, main_strategy)
    print(year)
    print(np.average(agent_outcome))
    print('\n')

2016
5.6


2017
6.4


2018
4.4




In [69]:
#######################################################
####################### set parameters ###########


## read in output from RF models

for year in [2016, 2017, 2018]:
    frame = pd.DataFrame()
    for pos in ['RB', 'WR', 'QB', 'TE']:
        temp = pd.read_csv('simulation_{}_{}.csv'.format(pos, year))
        temp['pos'] = pos
        frame = (frame
                 .append(temp)
                 .sort_values('Overall')
                 .reset_index(drop = True)
                )
        
    rounds = 7
    teams = 10

    main_strategy = {'maximizing_metric': 'Overall'
                    , 'sub_strategy': 'absolute'}

    agent_strategy = {'maximizing_metric': 'prediction_median'
                     , 'sub_strategy': 'absolute'}

    agent_outcome, simulation_outcome, pick_frame = run_simulation_one_year(teams, rounds, frame, agent_strategy, main_strategy)
    print(year)
    print(np.average(agent_outcome))
    print('\n')

2016
3.2


2017
3.5


2018
3.8




In [74]:
pick_frame.groupby('Team').sum()
simulation_outcome

,Team,Pick,Round,prediction_median,Overall,pts_next_year,finish
0,1.0,244.0,28.0,1497.386606,293.88,1426.8,6.0
1,2.0,245.0,28.0,1468.531748,302.41,1601.9,8.0
2,3.0,246.0,28.0,1440.339649,278.96,1705.4,10.0
3,4.0,247.0,28.0,1550.441902,277.94,1625.1,9.0
4,5.0,248.0,28.0,1443.511555,305.63,1162.8,1.0
5,6.0,249.0,28.0,1483.841932,323.80,1314.2,5.0
6,7.0,250.0,28.0,1361.155663,333.35,1470.8,7.0
7,8.0,251.0,28.0,1474.797368,332.61,1299.9,4.0
8,9.0,252.0,28.0,1547.252926,284.52,1254.5,2.0
9,10.0,253.0,28.0,1559.427388,392.73,1269.2,3.0


In [80]:
simulation_outcome
#remaining
#[print(i) for i in range(1, 10)]

pick_frame.loc[pick_frame.Team == 2]

,Pick,Round,Team,pos,Names,prediction_median,Overall,pts_next_year
1,2.0,1.0,2.0,,NaN,NaN,NaN,NaN
18,19.0,2.0,2.0,,NaN,NaN,NaN,NaN
21,22.0,3.0,2.0,,NaN,NaN,NaN,NaN
38,39.0,4.0,2.0,,NaN,NaN,NaN,NaN
41,42.0,5.0,2.0,,NaN,NaN,NaN,NaN
58,59.0,6.0,2.0,,NaN,NaN,NaN,NaN
61,62.0,7.0,2.0,,NaN,NaN,NaN,NaN


In [36]:
# pick_frame#.loc[pick_frame.Pick == pick].Team.iloc[0]
#
pick_frame.loc[pick_frame.Team == 10]
#remaining

,Pick,Round,Team,pos,Names,prediction_median,Overall,pts_next_year
9,10.0,1.0,10.0,WR,Julio Jones,292.064411,12.54,274.1
10,11.0,2.0,10.0,WR,Odell Beckham,231.514721,13.32,201.3
29,30.0,3.0,10.0,WR,Amari Cooper,224.430474,32.63,246.5
30,31.0,4.0,10.0,RB,Aaron Jones,141.331982,33.01,314.8
49,50.0,5.0,10.0,TE,O.J. Howard,121.655439,58.63,83.9
50,51.0,6.0,10.0,QB,Matt Ryan,265.315745,59.98,267.3
69,70.0,7.0,10.0,RB,Tevin Coleman,156.069576,74.83,135.4


,Pick,Round,prediction_median,Overall,pts_next_year
Team,,,,,
1.0,244.0,28.0,1545.664518,287.96,1516.5
2.0,245.0,28.0,1604.288271,366.83,1536.1
3.0,246.0,28.0,1503.894586,298.57,1563.9
4.0,247.0,28.0,1495.176878,278.02,1587.9
5.0,248.0,28.0,1451.105457,305.50,1368.6
6.0,249.0,28.0,1472.312780,300.73,983.2
7.0,250.0,28.0,1594.165736,304.53,1653.6
8.0,251.0,28.0,1553.548972,324.70,1308.7
9.0,252.0,28.0,1386.515676,331.70,1717.3


In [75]:
position = 'RB'
var = 'prediction_median'
sort_type = True if var == 'Overall' else False
pick_options = {}
for position in ['RB', 'QB', 'TE', 'WR']:
    temp1 = (remaining
     .loc[remaining.pos == position]
     .sort_values(var, ascending = sort_type)
     .reset_index(drop = True)
    )

    temp1['replacement_{}'.format(var)] = temp1[var].shift((-1)#* (teams - 1) * 2
                                                          )
    temp1['delta_{}'.format(var)] = abs(temp1[var] - temp1['replacement_{}'.format(var)])
    pick_options[position] = temp1.iloc[0]

In [77]:
pick_options
temp_pick = 0
for pos in pick_options:
    if pick_options[pos]['delta_{}'.format(var)] > temp_pick:
        chosen = pos
        temp_pick = pick_options[pos]['delta_{}'.format(var)]
chosen

'RB'

In [1]:
pick_options['RB']

NameError: name 'pick_options' is not defined